# Introduction to Big Data Modern Technologies course

## FINAL PROJECT: lab work
### Part 1. Object storage and database pipeline (Spark)

### 1. Libraries and credentials

In [ ]:
import os
import io
import sys
import json
import boto3
import logging
import psycopg2
import requests
import datetime
import multiprocessing
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.functions import udf, struct, countDistinct

In [ ]:
def access_data(file_path):
    with open(file_path) as file:
        access_data = json.load(file)
    return access_data

creds = access_data(file_path='access_bucket.json')
print(creds.keys())

### 2. Browse files at S3

In [ ]:
session = boto3.session.Session()
s3 = session.client(
    service_name='s3',
    aws_access_key_id=creds['aws_access_key_id'],
    aws_secret_access_key=creds['aws_secret_access_key'],
    endpoint_url='https://storage.yandexcloud.net'
)

In [ ]:
DATA_BUCKET = creds['name']

In [ ]:
all_files = [key['Key'] for key in s3.list_objects(Bucket=DATA_BUCKET)['Contents']]
print('files in storage:', all_files[:10]) # works only for num of files < 1000

### 3. Connection to database

##### <font color='blue'>ClickHouse</font>

In [ ]:
PATH_TO_JAR = '/home/jovyan/__DATA/IBDT_Spring_2024/topic_labs/jars/clickhouse-jdbc-0.4.0-shaded.jar'
CLICKHOUSE_JAR = f'file://{PATH_TO_JAR}'

In [ ]:
def execute_query(query, access_ch, data=None):
    url = 'https://{host}:{port}/'.format(
        host=access_ch['host'],
        port=access_ch['port']
    )
    params = {
        'database': access_ch['dbname'],
        'query': query.strip()
    }
    auth = {
        'X-ClickHouse-User': access_ch['user'],
        'X-ClickHouse-Key': access_ch['password']
    }
    rs = requests.post(
        url, 
        params=params, 
        headers=auth, 
        data=data,
        verify=f'/home/jovyan/__DATA/IBDT_Spring_2024/topic_labs/{access_ch["sslrootcert"]}'
    )
    return rs

In [ ]:
access_ch = access_data('access_ch.json')
print(access_ch.keys())

In [ ]:
query = '''
SELECT version()
'''
rs = execute_query(query, access_ch)
rs.text

##### <font color='green'>PostgreSQL</font>

In [ ]:
access_postgres = access_data('access_postgres.json')
print(access_postgres.keys())

In [ ]:
def send_query(query, access_postgres, res=False):
    result = None
    with psycopg2.connect(
        host=access_postgres['host'],
        port=access_postgres['port'],
        dbname=access_postgres['dbname'],
        user=access_postgres['user'],
        password=access_postgres['password'],
        target_session_attrs='read-write',
        sslmode='verify-full',
        sslrootcert=f'/home/jovyan/__DATA/IBDT_Spring_2024/topic_labs/{access_postgres["sslrootcert"]}'
        ) as conn:
            with conn.cursor() as cur:
                cur.execute(query)
                if res:
                    result = cur.fetchall()
    return result

In [ ]:
query = '''
SELECT version()
'''
send_query(query, access_postgres, res=True)

### 4. Data preprocessing with Spark

In [ ]:
# web UI for the Spark

def uiWebUrl(self):
    from urllib.parse import urlparse
    web_url = self._jsc.sc().uiWebUrl().get()
    port = urlparse(web_url).port
    return '{}proxy/{}/jobs/'.format(os.environ['JUPYTERHUB_SERVICE_PREFIX'], port)

SparkContext.uiWebUrl = property(uiWebUrl)

# Spark settings
conf = SparkConf()
conf.set('spark.master', 'local[*]')    # max 5 cores available, use `local[*]` for all cores
conf.set('spark.driver.memory', '16G')  # max 16 GB available
conf.set('spark.driver.maxResultSize', '4G')
conf.set('spark.driver.extraClassPath', CLICKHOUSE_JAR) # CH connect
sc = SparkContext(conf=conf)
spark = SparkSession(sc)

# Spark's access for object storage settings
spark._jsc.hadoopConfiguration().set('fs.s3a.access.key', creds['aws_access_key_id'])
spark._jsc.hadoopConfiguration().set('fs.s3a.secret.key', creds['aws_secret_access_key'])
spark._jsc.hadoopConfiguration().set('fs.s3a.impl','org.apache.hadoop.fs.s3a.S3AFileSystem')
spark._jsc.hadoopConfiguration().set('fs.s3a.multipart.size', '104857600')
spark._jsc.hadoopConfiguration().set('fs.s3a.block.size', '33554432')
spark._jsc.hadoopConfiguration().set('fs.s3a.threads.max', '256')
spark._jsc.hadoopConfiguration().set('fs.s3a.endpoint', 'http://storage.yandexcloud.net')
spark._jsc.hadoopConfiguration().set('fs.s3a.aws.credentials.provider', 
                                     'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider')

spark

#### 4.1. Read data with Spark

In [ ]:
sdf = spark.read.csv(
    f's3a://{DATA_BUCKET}/jhub_logs_prj.csv',
    sep=';', 
    header=True,
    multiLine=True, # if you have `\n` symbols
    escape="\""
)
sdf.printSchema()

In [ ]:
sdf.show(5)

#### 4.2. Kubernetes logs

In [ ]:
def row_info(rin):
    """
    Extracts names of:
      - docker image
      - id of the Jupyter application
      - name of the host, where Jupyter runs
    
    """
    img = rin[rin.find('container_image='):].split('\'')[1]
    hub = rin[rin.find('pod_name='):].split('\'')[1]
    host = rin[rin.find('host='):].split('\'')[1]
    return img, hub, host

In [ ]:
udf_row_info = udf(row_info, ArrayType(StringType()))

In [ ]:
sdf = sdf.withColumn('kubernetes_msg', udf_row_info('kubernetes'))
sdf.limit(5).toPandas()

In [ ]:
sdf = sdf.select(
    'date',
    F.col('kubernetes_msg')[0].alias('img'),
    F.col('kubernetes_msg')[1].alias('hub'),
    F.col('kubernetes_msg')[2].alias('host'),
    'log',
    'stream',
    'time'
)
sdf.limit(5).toPandas()

#### 4.3. JupyterHub logs

In [ ]:
def sq_brackets(sin):
    """
    Split log string amd extracts:
      - timestamp of the event
      - name of application
      - type of logs
      - code of event
      - description
    
    """
    try:
        s = sin.split('[', 1)[1].split(']')[0]
        msg = sin[len(s) + 2 :].strip()
        s = s.split()
        head = s[0]
        ts = ' '.join(s[1:3])
        svc = s[3]
        typ = s[4].split(':')[0]
        code = s[4].split(':')[1]
    except:
        head, ts, svc, typ, code = '', '', '', '', ''
        msg = sin
    return head, ts, svc, typ, code, msg

In [ ]:
udf_sq_brackets = udf(sq_brackets, ArrayType(StringType()))

In [ ]:
sdf = sdf.withColumn('log_msg', udf_sq_brackets('log'))
sdf.limit(5).toPandas()

In [ ]:
sdf = sdf.select(
    'img',
    'hub',
    'host',
    F.col('log_msg')[0].alias('head'),
    F.col('log_msg')[1].alias('timestamp'),
    F.col('log_msg')[2].alias('service'),
    F.col('log_msg')[3].alias('event_type'),
    F.col('log_msg')[4].alias('event_code'),
    F.col('log_msg')[5].alias('message')
)
sdf.limit(5).toPandas()

#### 4.4. Find users' activities

In [ ]:
def parce_users_activities(code, msg):
    """
    Ugly function.
    
    You may use dictionary to make it
    more pythonic or something else.
    
    """
    if code in ['43', '44']:
        user = msg.split()[-1]
        log = 'logged out'
    elif code in ['61', '85', '111']:
        user = msg.split()[3]
        log = 'spawning sever with advanced configuration option'
    elif code == '148':
        user = msg.split()[-1]
        log = 'user is running'
    elif code == '167':
        user = msg.split()[1]
        log = 'server is already active'
    if code == '238':
        user = msg.split()[-1]
        log = 'adding role for user'
    elif code in ['257', '333']:
        user = msg.split()[2]
        log = 'adding user to proxy'
    elif code in ['281', '359']:
        user = msg.split()[2]
        log = 'removing user from proxy'
    elif code == '361':
        user = msg.split()[1]
        log = 'user requested new auth token'
    elif code == '380':
        user = msg.split()[3]
        log = 'previous spawn failed'
    elif code in ['402', '394']:
        user = msg.split()[0]
        log = 'pending spawn'
    elif code == '567':
        user = msg.split('/')[4]
        log = 'stream closed while handling '
    elif code == '626':
        user = msg.split()[1]
        log = 'server is already started'
    elif code == '651':
        user = msg.split('-')[2]
        log = 'creating oauth client for user'
    elif code in ['664', '749']:
        user = msg.split()[1]
        log = 'server is ready'
    elif code == '681':
        user = msg.split()[0].replace('\'s', '')
        log = 'server failed to start'
    elif code == '689':
        user = msg.split()[3].replace('\'s', '')
        log = 'unhandled error starting with timeout'
    elif code == '738':
        user = msg.split()[0].replace(',', '').replace('\'s', '')
        log = 'server never showed up and giving up'
    elif code in ['757', '810']:
        user = msg.split()[-1]
        log = 'logged in'
    elif code in ['904', '963']:
        user = msg.split()[1]
        log = 'server took time to start'
    elif code in ['1067', '2022']:
        user = msg.split()[1]
        log = 'user server stopped with exit code 1'
    elif code in ['1110', '1170']:
        user = msg.split()[1]
        log = 'server took time to stop'
    elif code in ['1143', '1203']:
        user = msg.split()[1].replace(':', '')
        log = 'server is slow to stop'
    elif code in ['1344', '1409']:
        user = msg.split('/')[3]
        log = 'failing suspected api request to not-running server'
    elif code in ['1143', '1203']:
        user = msg.split()[1].replace('/', '')
        log = 'server is slow to stop'
    elif code == '1415':
        user = msg.split()[-1]
        log = 'admin requesting spawn on behalf'
    elif code == '1437':
        user = msg.split()[1]
        log = 'user requested server which user do not own'
    elif code == '1840':
        user = msg.split()[4].replace('jupyter-', '').replace(',', '')
        log = 'attempting to create pod with timeout'
    elif code == '1857':
        user = msg.split()[3].replace('jupyter-', '').replace(',', '')
        log = 'found existing pod and attempting to kill'
    elif code == '1861':
        user = msg.split()[2].replace('jupyter-', '').replace(',', '')
        log = 'killed pod and will try starting singleuser pod again'
    elif code in ['1875', '2469', '2509']:
        user = msg.split()[4].replace('claim-', '').replace('jupyter-', '').replace(',', '')
        log = 'attempt to create pvc with timeout'
    elif code in ['1887', '2525']:
        user = msg.split()[1].replace('claim-', '')
        log = 'pvc already exists'
    elif code in ['1961', '2044']:
        user = msg.split()[1].replace('jupyter-', '')
        log = 'restarting pod reflector'
    elif code in ['1997', '2780']:
        user = msg.split('-')[-1]
        log = 'deleting pod'
    elif code == '2069':
        user = msg.split()[0].replace(',', '')
        log = 'user does not appear to be running and shutting it down'  
    elif code in ['2077', '2504']:
        user = msg.split()[0]
        log = 'still running'
    elif code == '2085':
        user = msg.split()[0]
        log = 'server appears to have stopped while the hub was down'
    elif code == '2170':
        user = msg.split('-')[-1]
        log = 'deleting oauth client'
    else:
        user, log = '', ''
    return user, log

In [ ]:
udf_parce_users_activities = udf(parce_users_activities, ArrayType(StringType()))

In [ ]:
sdf = sdf.withColumn('user_act', udf_parce_users_activities(sdf['event_code'], sdf['message']))
sdf.limit(5).toPandas()

In [ ]:
sdf = sdf.select(
    'timestamp',
    'hub',
    'img',
    'host',
    'event_code',
    'event_type',
    F.col('user_act')[1].alias('log'),
    F.col('user_act')[0].alias('user')
)
sdf.limit(5).toPandas()

In [ ]:
sdf = sdf.filter(sdf.user != '')
sdf = sdf.withColumn(
    'timestamp',
    F.to_timestamp('timestamp', 'yyyy-MM-dd HH:mm:ss.SSS')
)
sdf.limit(5).toPandas()

### 5. Normalize data and write to database

#### 5.1. Users table

In [ ]:
logins = sdf.select('user').distinct().collect()
print(len(logins))
logins = [list(x)[0] for x in logins]
logins[:5]

In [ ]:
!pip install names

In [ ]:
import names

In [ ]:
users = []
for login in logins:
    user = {}
    user['login'] = login
    user['name'] = names.get_full_name()
    user['email'] = login + '@gsom.spbu.ru'
    users.append(user)

In [ ]:
rdd = sc.parallelize([users])
sdf_users = spark.read.json(rdd)
sdf_users.printSchema()

In [ ]:
sdf_users.show(5)

##### <font color='blue'>ClickHouse</font>

In [ ]:
query = '''
DROP TABLE db1.users;
'''
rs = execute_query(query, access_ch)
rs.text

In [ ]:
query = '''
CREATE TABLE IF NOT EXISTS db1.users (
    email String,
    login String,
    name String
) ENGINE = MergeTree
ORDER BY email;
'''
rs = execute_query(query, access_ch)
rs

In [ ]:
URL_CH = 'jdbc:clickhouse://{}:{}/{}'.format(
    access_ch['host'],
    access_ch['port'],
    access_ch['dbname']
)
DRIVER_CH = 'com.clickhouse.jdbc.ClickHouseDriver'

In [ ]:
table_name = 'users'

sdf_users.write \
    .format('jdbc') \
    .mode('append') \
    .option('url', URL_CH) \
    .option('user', access_ch['user']) \
    .option('password', access_ch['password']) \
    .option('dbtable', table_name) \
    .option('driver', DRIVER_CH) \
    .option('ssl', 1) \
    .option('sslmode', 'strict') \
    .option('sslrootcert', f'/home/jovyan/__DATA/IBDT_Spring_2024/topic_labs/{access_ch["sslrootcert"]}') \
    .save()

In [ ]:
# test query
query = '''
SELECT * FROM db1.users LIMIT 5;
'''
rs = execute_query(query, access_ch)
print(rs.text)

##### <font color='green'>PostgreSQL</font>

In [ ]:
query = '''
DROP TABLE IF EXISTS users
'''
send_query(query, access_postgres)

In [ ]:
query = '''
CREATE TABLE IF NOT EXISTS users (
    email varchar(128),
    login varchar(32),
    name varchar(128)
);
'''
send_query(query, access_postgres)

In [ ]:
URL_PG = 'jdbc:postgresql://{}:{}/{}'.format(
    access_postgres["host"],
    access_postgres["port"],
    access_postgres["dbname"]
)
DRIVER_PG = 'org.postgresql.Driver'

In [ ]:
sdf_users.write\
    .mode('append') \
    .format('jdbc') \
    .option('url', URL_PG) \
    .option('dbtable', table_name) \
    .option('user', access_postgres['user']) \
    .option('password', access_postgres['password']) \
    .option('driver', DRIVER_PG) \
    .option('ssl', True) \
    .option('sslmode', 'require') \
    .save()

In [ ]:
query = '''
SELECT * FROM users LIMIT 5;
'''
send_query(query, access_postgres, res=True)

#### 5.2. JupyterHub instances table

In [ ]:
sdf_instances = sdf.select(
    'hub',
    'img',
    'host'
)
sdf_instances = sdf_instances.dropDuplicates()

##### <font color='blue'>ClickHouse</font>

In [ ]:
query = '''
DROP TABLE db1.instances;
'''
rs = execute_query(query, access_ch)
rs.text

In [ ]:
query = '''
CREATE TABLE IF NOT EXISTS db1.instances (
    hub String,
    img String,
    host String
) ENGINE = MergeTree
ORDER BY hub;
'''
rs = execute_query(query, access_ch)
rs

In [ ]:
table_name = 'instances'

sdf_instances.write \
    .format('jdbc') \
    .mode('append') \
    .option('url', URL_CH) \
    .option('user', access_ch['user']) \
    .option('password', access_ch['password']) \
    .option('dbtable', table_name) \
    .option('driver', DRIVER_CH) \
    .option('ssl', 1) \
    .option('sslmode', 'strict') \
    .option('sslrootcert', f'/home/jovyan/__DATA/IBDT_Spring_2024/topic_labs/{access_ch["sslrootcert"]}') \
    .save()

In [ ]:
# test query
query = '''
SELECT * FROM db1.instances LIMIT 5;
'''
rs = execute_query(query, access_ch)
print(rs.text)

##### <font color='green'>PostgreSQL</font>

In [ ]:
query = '''
DROP TABLE IF EXISTS instances
'''
send_query(query, access_postgres)

In [ ]:
query = '''
CREATE TABLE IF NOT EXISTS instances (
    hub varchar(64),
    img varchar(64),
    host varchar(64)
);
'''
send_query(query, access_postgres)

In [ ]:
sdf_instances.write\
    .mode('append') \
    .format('jdbc') \
    .option('url', URL_PG) \
    .option('dbtable', table_name) \
    .option('user', access_postgres['user']) \
    .option('password', access_postgres['password']) \
    .option('driver', DRIVER_PG) \
    .option('ssl', True) \
    .option('sslmode', 'require') \
    .save()

In [ ]:
query = '''
SELECT * FROM instances LIMIT 5;
'''
send_query(query, access_postgres, res=True)

#### 5.3. JupyterHub logs table

In [ ]:
sdf_logs = sdf.select(
    'timestamp',
    'hub',
    'event_code',
    'event_type',
    'log',
    F.col('user').alias('login')
)
sdf_logs = sdf_logs.dropDuplicates()

##### <font color='blue'>ClickHouse</font>

In [ ]:
query = '''
DROP TABLE db1.logs;
'''
rs = execute_query(query, access_ch)
rs.text

In [ ]:
query = '''
CREATE TABLE IF NOT EXISTS db1.logs (
    timestamp DateTime,
    hub String,
    event_code Int32,
    event_type String,
    log String,
    login String
) ENGINE = MergeTree
ORDER BY hub;
'''
rs = execute_query(query, access_ch)
rs

In [ ]:
table_name = 'logs'

sdf_logs.write \
    .format('jdbc') \
    .mode('append') \
    .option('url', URL_CH) \
    .option('user', access_ch['user']) \
    .option('password', access_ch['password']) \
    .option('dbtable', table_name) \
    .option('driver', DRIVER_CH) \
    .option('ssl', 1) \
    .option('sslmode', 'strict') \
    .option('sslrootcert', f'/home/jovyan/__DATA/IBDT_Spring_2024/topic_labs/{access_ch["sslrootcert"]}') \
    .save()

In [ ]:
# test query
query = '''
SELECT * FROM db1.logs LIMIT 5;
'''
rs = execute_query(query, access_ch)
print(rs.text)

##### <font color='green'>PostgreSQL</font>

In [ ]:
query = '''
DROP TABLE IF EXISTS logs
'''
send_query(query, access_postgres)

In [ ]:
query = '''
CREATE TABLE IF NOT EXISTS logs (
    timestamp timestamp,
    hub varchar(64),
    event_code varchar(32),
    event_type varchar(64),
    log text,
    login varchar(32)
);
'''
send_query(query, access_postgres)

In [ ]:
sdf_logs.write\
    .mode('append') \
    .format('jdbc') \
    .option('url', URL_PG) \
    .option('dbtable', table_name) \
    .option('user', access_postgres['user']) \
    .option('password', access_postgres['password']) \
    .option('driver', DRIVER_PG) \
    .option('ssl', True) \
    .option('sslmode', 'require') \
    .save()

In [ ]:
query = '''
SELECT * FROM logs LIMIT 5;
'''
send_query(query, access_postgres, res=True)

### 6. How to get data back

#### 6.1. Requests and psycopg2 libraries

##### <font color='blue'>ClickHouse</font>

In [ ]:
query = '''
SELECT \
    ls.timestamp, \
    ls.event_code, \
    us.login, \
    us.name, \
    us.email, \
    ls.log \
FROM logs AS ls \
LEFT JOIN users AS us ON ls.login = us.login \
LIMIT 5;
'''
rs = execute_query(query, access_ch)

In [ ]:
rs

In [ ]:
rs.text

In [ ]:
rs.content

In [ ]:
df = pd.read_csv(io.StringIO(rs.text), sep='\t', header=None)
df

##### <font color='green'>PostgreSQL</font>

In [ ]:
query = '''
SELECT \
    ls.timestamp, \
    ls.event_code, \
    us.login, \
    us.name, \
    us.email, \
    ls.log \
FROM logs AS ls \
LEFT JOIN users AS us ON ls.login = us.login \
LIMIT 5;
'''
send_query(query, access_postgres, res=True)

#### 6.2. Spark connection

##### <font color='blue'>ClickHouse</font>

In [ ]:
table_name = 'users'

sdf_test = spark.read \
    .format('jdbc') \
    .option('url', URL_CH) \
    .option('dbtable', table_name) \
    .option('user', access_ch['user']) \
    .option('password', access_ch['password']) \
    .option('driver', DRIVER_CH) \
    .option('ssl', 1) \
    .option('sslmode', 'strict') \
    .option('sslrootcert', f'/home/jovyan/__DATA/IBDT_Spring_2024/topic_labs/{access_ch["sslrootcert"]}') \
    .load()

In [ ]:
sdf_test.show(5)

##### <font color='green'>PostgreSQL</font>

In [ ]:
sdf_test = spark.read \
    .format('jdbc') \
    .option('url', URL_PG) \
    .option('dbtable', table_name) \
    .option('user', access_postgres['user']) \
    .option('password', access_postgres['password']) \
    .option('driver', DRIVER_PG) \
    .option('ssl', True) \
    .option('sslmode', 'require') \
    .load()

In [ ]:
sdf_test.show(5)

#### 6.3. ClickHouse connect

[Python packages](https://pypi.org/project/clickhouse-connect/) for connecting Python to ClickHouse.

In [ ]:
!pip install clickhouse-connect

In [ ]:
import clickhouse_connect

In [ ]:
client = clickhouse_connect.get_client(
    host=access_ch['host'], 
    username=access_ch['user'], 
    password=access_ch['password'],
    port=access_ch['port'],
    verify=f'/home/jovyan/__DATA/IBDT_Spring_2024/topic_labs/{access_ch["sslrootcert"]}'
)

In [ ]:
client

##### 6.3.1. Queries

In [ ]:
client.command('SELECT COUNT(*) FROM db1.users')

In [ ]:
result = client.query('SELECT * FROM db1.users LIMIT 5')
result.result_rows

In [ ]:
parameters = {
    'table1': 'logs',
    'table2': 'users',
    'key': 'login',
    'user': 'vgarshin@gsom.spbu.ru'
}
result = client.query(
    "SELECT COUNT(*) FROM db1.{table1:Identifier} AS ls \
    LEFT JOIN db1.{table2:Identifier} AS us ON ls.{key:Identifier} = us.{key:Identifier} \
    WHERE us.email = {user: String}",
    parameters=parameters
)
result.result_rows

##### 6.3.2. Insert data

In [ ]:
row1 = ['test1@gsom.spbu.ru', 'test1', 'Test Number One']
row2 = ['test2@gsom.spbu.ru', 'test2', 'Test Number Two']
data = [row1, row2]
data

In [ ]:
client.insert(
    'db1.users', 
    data, 
    column_names=['email', 'login', 'name']
)

In [ ]:
result = client.query(
    "SELECT * FROM db1.users WHERE email='test1@gsom.spbu.ru'", 
    parameters=parameters
)
result.result_rows

In [ ]:
parameters = {
    'table': 'users', 
    'v1': 'Test%'
}
result = client.query(
    'SELECT * FROM db1.{table:Identifier} WHERE name LIKE {v1:String}', 
    parameters=parameters
)
result.result_rows